In [3]:
import numpy as np

# Define Pauli matrices
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Define Hadamard gate
H = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])

# Define qubits
qubit0 = np.array([1, 0])  # |0>
qubit1 = np.array([1, 0])  # |0>

# Apply Hadamard gate to qubit 0
qubit1 = np.dot(H, qubit1)

# Print state of qubit 0 after Hadamard gate
print("State of qubit 0 after Hadamard gate:")
print(qubit1)


State of qubit 0 after Hadamard gate:
[0.70710678 0.70710678]


In [4]:
import numpy as np

classA_data = np.loadtxt('classA_train.dat', delimiter='\t')
classB_data = np.loadtxt('classB_train.dat', delimiter='\t')
np.set_printoptions(precision=17, suppress=False)



In [126]:
print("Class A Data:")
print(classA_data[0])
print("\nClass B Data:")
print(classB_data[0])

Class A Data:
[-0.0213693207939733  1.0163426091550307  1.076851867211415 ]

Class B Data:
[0.080523695123608   0.9765466754734927  0.01419055980597866]


In [135]:
from sklearn.preprocessing import MinMaxScaler 
#normalize to [0,1]
scaler=MinMaxScaler(feature_range=(-1, 1))

trainA_norm=scaler.fit_transform(classA_data)
trainB_norm=scaler.fit_transform(classB_data)
print(trainA_norm[0])

[-0.9040470529080318  0.8940930776226766  1.                ]


In [146]:
#map to angles (angle encoding)
angles_A=classA_data * np.pi/2
angles_B=classB_data * np.pi/2
print(angles_A[0])

[-0.03356685060927506  1.5964672372258635   1.6915149575179163 ]


In [147]:
#Define the Ry gate
def apply_Ry(state, theta):
    cos=np.cos(theta/2)
    sin=np.sin(theta/2)
    Ry=np.array([[cos, -sin], [sin, cos]], dtype=complex)
    new_state = np.dot(Ry, state)
    return new_state / np.linalg.norm(new_state)
    
#define the CNOT gate
def apply_CNOT(control, target):
    combined_state = np.kron(control, target)
    CNOT = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0]
    ], dtype=complex)
    result = np.dot(CNOT, combined_state).reshape((2, 2))
    result = result / np.linalg.norm(result)
    return result



In [148]:
#circuit design

#ANGLE ENCODING 
def encode(angles):
    qubits=angles.shape[1]
    final_states=[]
    
    #for each row of the data
    for angle in angles:
        #initialize qubits to |0>
        q_states=[np.array([1,0], dtype=complex) for _ in range(qubits)]
    
        #for each quantum state of the qbit apply the Ry gate
        for i in range(qubits):
            q_states[i]=apply_Ry(q_states[i], angle[i])
           
        final_states.append(q_states)
    return final_states

finalstates_A = encode(angles_A)
states_B=encode(angles_B)
print(len(final_states_A))

# The gate rotate the quantum states in a way that encodes the information from your data into the quantum state of the qubits
print("Encoded Quantum States for First Row of Data:")
#state for each qubit of 1st row

for st in states_A[0]:
    print(st[0] ,st[1])
    

20
Encoded Quantum States for First Row of Data:
(0.9994366861651128+0j) (-0.033560547482095686+0j)
(-0.025668091020849194+0j) (0.9996705202732277+0j)
(-0.12042563910382104+0j) (0.9927223506330642+0j)


In [149]:
qubits=len(states_A[0])
print(qubits)



3


In [150]:
#circuit design
def apply_Rx(state, theta):
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    Rx = np.array([
        [cos, -1j * sin],
        [-1j * sin, cos]
    ], dtype=complex)
    new_state=np.dot(Rx, state)
    return new_state / np.linalg.norm(new_state)



#VARIATIONAL MODEL
def circuit(states, parameters):
    qubits=len(states[0])
    finals=[]
    for s in states:
        new_state = [np.copy(q) for q in s]  # Copy the state to avoid in-place modifications
        for i in range(qubits):
            new_state[i] = apply_Rx(new_state[i], parameters[i])
            new_state[i] = apply_Ry(new_state[i], parameters[i + qubits])

                # Apply CNOT gate
        for i in range(qubits - 1):  # Loop over all qubits except the last one
            control = new_state[i]
            target = new_state[i + 1]
            combined_state = apply_CNOT(control, target)
            new_state[i] = combined_state[:, 0]  # Extract the new state of control qubit
            new_state[i + 1] = combined_state[:, 1]         
        finals.append(new_state)
    return finals

            
    


In [151]:
num_qubits = 3
num_params = 2 * num_qubits  # Each qubit has Rx and Ry

# Choose the standard deviation for parameter initialization
std_dev = 0.1

# Initialize parameters from a normal distribution
initial_parameters = np.random.normal(loc=0, scale=std_dev, size=num_params)
finalsA=circuit(states_A, initial_parameters)
finalsB=circuit(states_B, initial_parameters)

print("initial parameters: ", initial_parameters[4])

print("First row of data after cicuit with random theta:")
for st in (finalsA[0]):
    print(st[0], st[1])

initial parameters:  0.12647127652330267
First row of data after cicuit with random theta:
(-0.08878127467934313-0.009533453352265983j) (0.012762166470103425+0.010217696549871601j)
(-0.06491377419794246+0.0322025067460508j) (-0.0010461960925231787-0.0010298731221924885j)
(0.9973461325925249-0.00690838189260682j) (0.00010111081834762885+3.3951429317686376e-05j)


In [154]:
M = np.array([[1, 0], [0, -1]], dtype=complex)
basis_state0=np.array([1 , 0], dtype=complex)    #|0> state
basis_state1=np.array([0 , 1 ], dtype=complex)   #|1> state

qubits=3;

def caclf(qstate):
    #print(qstate[0]  ,   qstate[1])
    alpha=qstate[0]
    beta=qstate[1]
    norm_factor = np.sqrt(np.abs(alpha)**2 + np.abs(beta)**2)
    alpha_norm = alpha / norm_factor
    beta_norm = beta / norm_factor
    ket=alpha_norm*basis_state0 + beta_norm*basis_state1
    #pr1=np.abs(alpha_norm)**2
    #prm1=np.abs(beta_norm)**2
    bra = np.conj(ket).T
    m_res = np.dot(M, ket)
    # Calculate f_theta(x) as bra * Z * ket
    f_theta = np.dot(bra, m_res)
    return np.real(f_theta)


def classify(states_allq):
    f_theta_prod=1
    for i in states_allq:
        #print("ΓΙΑ καθε qbit")
        #print(i)
        res=caclf(i)
        f_theta_prod*=res

    
    #1 if classA 0 if classB 
    print("f theta is:")
    print(f_theta_prod)
    return "classA" if f_theta_prod > 0 else "classB"

            


   
#def f_for_each_qbit(state)
predA=[]
predB=[]
for finalA in finalsA:
    pr = classify(finalA)
    predA.append(pr)

for finalB in finalsB:
    prb=classify(finalB)
    predB.append(prb)
    
print(predA)
print(predB)



f theta is:
0.9343624425642052
f theta is:
0.990159878215815
f theta is:
0.9854713981680558
f theta is:
0.879493880865435
f theta is:
0.9181046256332737
f theta is:
-0.9128208744615527
f theta is:
0.9352682613930131
f theta is:
-0.939398931055979
f theta is:
0.9566842868093558
f theta is:
-0.4940801525730776
f theta is:
0.9854128574256121
f theta is:
0.3993297405007868
f theta is:
-0.9361167183287273
f theta is:
0.9931176757898662
f theta is:
0.812918666032282
f theta is:
-0.9851775587429044
f theta is:
-0.9490496949588119
f theta is:
0.49477434907932605
f theta is:
-0.9408712103338055
f theta is:
-0.2106299239587302
f theta is:
-0.5244793954050001
f theta is:
0.16153311780603086
f theta is:
-0.284702633162722
f theta is:
0.902043336548429
f theta is:
0.0006978423093268518
f theta is:
-0.0033794427951683225
f theta is:
-0.03230395032440371
f theta is:
0.0019181968544088846
f theta is:
-0.02540166567359864
f theta is:
0.007348264405418088
f theta is:
0.5458805350270003
f theta is:
0.065

In [19]:
#: Collapse each qubit’s wave function to obtain one of the quantum states of the computational basis.

basis_state0=np.array([1 + 0j, 0 + 0j])    #|0> state
basis_state1=np.array([0 + 0j, 1 + 0j])   #|1> state


def calculate_probabilities(qubit_state):
    # Define |0⟩ and |1⟩ states
    basis_state0 = np.array([1 + 0j, 0 + 0j])  # |0⟩ state
    basis_state1 = np.array([0 + 0j, 1 + 0j])  # |1⟩ state
    
    # Calculate the probabilities of measuring |0⟩ and |1⟩ states
    prob_1 = np.abs(np.dot(qubit_state, basis_state1)) ** 2
    prob_0 = 1-prob_1
    
    return prob_0, prob_1
binaries=[]
# Iterate through each final quantum state in finalsA
for final_state in finalsA:
    # Iterate through each qubit in the final quantum state
    measurements =[]
    for qubit_state in final_state:
        # Calculate probabilities of measuring |0⟩ and |1⟩ states for the current qubit
        prob_0, prob_1 = calculate_probabilities(qubit_state)
        
       # print("Prob0:", prob_0)
        #print("Prob1:", prob_1)
        
        # Generate random outcome based on probabilities
        measurement = np.random.choice([0, 1], p=[prob_0, prob_1])
        measurements.append(str(measurement))
    bmeasur=''.join(measurements)
    binaries.append(bmeasur)

    

    



In [20]:
from collections import Counter

# Example measurement outcomes (binary strings)

# Count occurrences of each unique outcome
outcome_counts = Counter(binaries)

# Total number of measurements
total_measurements = len(binaries)

# Calculate probability of each outcome
outcome_probabilities = {outcome: count / total_measurements for outcome, count in outcome_counts.items()}

# Print probabilities
for outcome, probability in outcome_probabilities.items():
    print(f"Outcome: {outcome}, Probability: {probability}")


Outcome: 000, Probability: 0.4
Outcome: 100, Probability: 0.25
Outcome: 101, Probability: 0.15
Outcome: 010, Probability: 0.2


In [327]:
def determine_parity(probabilities):
    num_ones = sum(int(outcome) for outcome in probabilities)
    return num_ones % 2 == 0  # True if even parity, False if odd parity

# Extract labels based on parity
def extract_labels(probabilities):
    parity = determine_parity(probabilities)
    return 'even' if parity else 'odd'

# Example: Extract labels from basis state probabilities
labels = extract_labels(outcome_probabilities)
print("Labels:", labels)

Labels: even


In [103]:
print(len(final_states[0]))

3
